# ADMM

In [1]:
# Importing the required libraries
import pandas as pd
import numpy as np
import json
import os

In [2]:
pd.set_option("display.expand_frame_repr", True)

In [3]:
# Importing the datasets
X_train = pd.read_csv("dataset/X_train.csv")
t_train = pd.read_csv("dataset/t_train.csv")

In [4]:
# Converting the dataframes to numpy arrays
X_train = X_train.values
t_train = t_train.values

In [5]:
# Reshaping the t_train array
t_train = t_train.reshape((t_train.shape[0],))

In [6]:
# Adding the bias term to the X_train array
X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))

In [7]:
# Printing the shapes
print("Shape of X_train: {}".format(X_train.shape))
print("Shape of t_train: {}".format(t_train.shape))

Shape of X_train: (3196, 10)
Shape of t_train: (3196,)


In [8]:
# Defining the necessary variables
n_agents = 3
n_parameters = 10
n_iterations = 700

# Defining the epsilon value to avoid division by zero in cost function
epsilon = 0.00001

In [9]:
# Defining a function to save the results in a json file
def create_empty_json_file(filename):
    # Checking if the file exists
    if not os.path.isfile(f"results/{filename}"):
        # Creating the file
        with open(f"results/{filename}", "w") as json_file:
            json.dump([], json_file)

def save_results(results, filename):
    # Checking if the file exists
    create_empty_json_file(filename)

    # Opening the JSON file in read mode
    with open(f"results/{filename}", "r") as json_file:
        # Loading the content of the file
        json_content = json.load(json_file)

    # Adding the new results to the list
    json_content.append(results)

    # Opening the JSON file in write mode
    with open(f"results/{filename}", "w") as json_file:
        # Saving the results in the JSON file
        json.dump(json_content, json_file)

In [10]:
# Defining the sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [11]:
# Defining the cost function
def cost_function(X, t, weight_parameters, n_agents=0):

    if n_agents == 0:
        h = sigmoid(np.matmul(X, weight_parameters))
        N = X.shape[0]
        cost = (np.dot(-t.T, np.log(h + epsilon)) - np.dot((1 - t.T), np.log(1 - h + epsilon))) / N
        return cost

    cost = np.zeros([n_agents, 1])
    for i in range(0, n_agents):
        data_for_agent = int(X.shape[0] / n_agents)
        X_i = X[data_for_agent * i : data_for_agent * (i+1), :]
        t_i = t[data_for_agent * i : data_for_agent * (i+1)]

        cost[i, :] = cost_function(X_i, t_i, weight_parameters[:, i])
    return cost.reshape(3)

In [12]:
# Defining the function to update the local variable
def update_local_variable(X_train, t_train, weight_parameters, z, u, rho):

    data_for_agent = int(X_train.shape[0] / n_agents)
    n_iterations = 700
    for i in range(0, n_agents):

        X_i = X_train[data_for_agent * i : data_for_agent * (i+1), :]
        t_i = t_train[data_for_agent * i : data_for_agent * (i+1)]

        n_samples = X_i.shape[0]

        weight_parameters_gradient = np.zeros([n_parameters, n_iterations])
        weight_parameters_gradient[:, 0] = weight_parameters[:, i]

        cost_function_values = []
        cost_function_values.append(cost_function(X_train, t_train, weight_parameters[:, i]))

        for j in range(0, n_iterations-1):

            s = sigmoid(np.dot(X_i, weight_parameters[:, i]))
            lr = n_agents / data_for_agent

            weight_parameters_gradient[:, j+1] = weight_parameters_gradient[:, j] - (lr) * (1 / n_samples) * (np.dot(X_i.T, (s - t_i))) + rho * (weight_parameters_gradient[:, j] - z + u[:, i])

        weight_parameters[:, i] = weight_parameters_gradient[:, -1]
    return weight_parameters

In [13]:
# Defining the function to update the global variable
def update_global_variable(weight, u):

    average_weights = np.zeros(n_parameters)
    average_u = np.zeros(n_parameters)

    for i in range(0, n_agents):
        average_weights += weight[:, i]
        average_u += u[:, i]

    return (average_weights + average_u) / n_agents

In [14]:
# Defining the function to update the dual variable
def update_dual_variable(u, weight_parameters, z):

    current_u = np.zeros([n_parameters, n_agents])

    for i in range(n_agents):
        current_u[:, i] = u[:, i] + weight_parameters[:, i] - z

    return current_u

In [15]:
# Defining the ADMM function
def ADMM(X_train, t_train):
    # Initializing the weight parameters, z and u
    weight_parameters = np.zeros([n_parameters, n_iterations, n_agents])
    z = np.zeros([n_parameters, n_iterations])
    u = np.zeros([n_parameters, n_iterations, n_agents]) - 1

    # Assigning the initial values of the weight parameters and u randomly
    weight_parameters[:, 0, :] = np.random.randn(n_parameters, n_agents)
    u[:, 0, :] = np.random.randn(n_parameters, n_agents)

    # Updating the value of z
    z[:, 0] = update_global_variable(weight_parameters[:, 0, :], u[:, 0, :])

    rho_list = [0, 0.00001, 0.0001]
    for rho in rho_list:

        print(f"Rho: {rho}")
        
        # Initializing the cost values matrix
        logistic_cost_values = np.zeros([n_agents, n_iterations])
        logistic_cost_values[:, 0] = cost_function(X_train, t_train, weight_parameters[:, 0, :], n_agents)

        for i in range(0, n_iterations-1):

            print(f"Iterazione n: {i}")

            weight_parameters[:, i+1, :] = update_local_variable(X_train, t_train, weight_parameters[:, i, :], z[:, i], u[:, i, :], rho)

            z[:, i+1] = update_global_variable(weight_parameters[:, i+1, :], u[:, i, :])

            u[:, i+1, :] = update_dual_variable(u[:, i, :], weight_parameters[:, i+1, :], z[:, i+1])

            logistic_cost_values[:, i+1] = cost_function(X_train, t_train, weight_parameters[:, i+1, :], n_agents)

        cost = []
        for j in range(0, n_iterations):
            cost.append(logistic_cost_values[:, j].tolist())

        # Saving the results in a json file
        results = {
            "rho": rho,
            "cost_function_values": cost,
            "optimal_parameters_values": [weight_parameters[:, n_iterations-1, j].tolist() for j in range(0, n_agents)],
            "z": z[:, -1].tolist()
        }
        save_results(results, "ADMM.json")

In [16]:
# Calling the ADMM function
ADMM(X_train, t_train)

Rho: 0
Iterazione n: 0
Iterazione n: 1
Iterazione n: 2
Iterazione n: 3
Iterazione n: 4
Iterazione n: 5
Iterazione n: 6
Iterazione n: 7
Iterazione n: 8
Iterazione n: 9
Iterazione n: 10
Iterazione n: 11
Iterazione n: 12
Iterazione n: 13
Iterazione n: 14
Iterazione n: 15
Iterazione n: 16
Iterazione n: 17
Iterazione n: 18
Iterazione n: 19
Iterazione n: 20
Iterazione n: 21
Iterazione n: 22
Iterazione n: 23
Iterazione n: 24
Iterazione n: 25
Iterazione n: 26
Iterazione n: 27
Iterazione n: 28
Iterazione n: 29
Iterazione n: 30
Iterazione n: 31
Iterazione n: 32
Iterazione n: 33
Iterazione n: 34
Iterazione n: 35
Iterazione n: 36
Iterazione n: 37
Iterazione n: 38
Iterazione n: 39
Iterazione n: 40
Iterazione n: 41
Iterazione n: 42
Iterazione n: 43
Iterazione n: 44
Iterazione n: 45
Iterazione n: 46
Iterazione n: 47
Iterazione n: 48
Iterazione n: 49
Iterazione n: 50
Iterazione n: 51
Iterazione n: 52
Iterazione n: 53
Iterazione n: 54
Iterazione n: 55
Iterazione n: 56
Iterazione n: 57
Iterazione n: 58


/var/folders/3w/b4rc1c293w15rt3bzm87ymhm0000gn/T/ipykernel_5652/346045589.py:3: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Iterazione n: 108
Iterazione n: 109
Iterazione n: 110
Iterazione n: 111
Iterazione n: 112
Iterazione n: 113
Iterazione n: 114
Iterazione n: 115
Iterazione n: 116
Iterazione n: 117
Iterazione n: 118
Iterazione n: 119
Iterazione n: 120
Iterazione n: 121
Iterazione n: 122
Iterazione n: 123
Iterazione n: 124
Iterazione n: 125
Iterazione n: 126
Iterazione n: 127
Iterazione n: 128
Iterazione n: 129
Iterazione n: 130
Iterazione n: 131
Iterazione n: 132
Iterazione n: 133
Iterazione n: 134
Iterazione n: 135
Iterazione n: 136
Iterazione n: 137
Iterazione n: 138
Iterazione n: 139
Iterazione n: 140
Iterazione n: 141
Iterazione n: 142
Iterazione n: 143
Iterazione n: 144
Iterazione n: 145
Iterazione n: 146
Iterazione n: 147
Iterazione n: 148
Iterazione n: 149
Iterazione n: 150
Iterazione n: 151
Iterazione n: 152
Iterazione n: 153
Iterazione n: 154
Iterazione n: 155
Iterazione n: 156
Iterazione n: 157
Iterazione n: 158
Iterazione n: 159
Iterazione n: 160
Iterazione n: 161
Iterazione n: 162
Iterazione